In [1]:
import math

def convert_milli_watt_to_dbm(x):
    return 10 * math.log10(x)

def convert_dbm_to_milli_watt(x):
    return math.pow(10, x/10)

def convert_ratio_to_db(x):
    return 10 * math.log10(x)

## Requirements


- The `dBi` unit represents the gain of an antenna in decibels relative to an isotropic radiator

- The `noise figure` is a measure of how much noise is added by a device or component, such as an amplifier or a receiver, to the signal passing through it
    - Lower is better
    - Reduces the SNR
    - NF(dB) = SNR_in(dB) - SNR_out(dB)
    - SNR_out(dB) = SNR_in(dB) - NF(dB)
    
- The `receive sensitivity` is the minimum power level of an incoming signal that the receiver can detect and correctly decode.
    - Higher is better

- The `CQI` is the Channel Quality Indicator

- The `MCS` is the Modulation and Coding Scheme

In [2]:
bandwidth_hz = 20e6
frequency_hz = 2e9

gnb_transmit_power_milli_watt = 1e3
gnb_cable_loss_db = 0.5
gnb_noise_figure_db = 5
gnb_receive_sensitivity_dbm = -102
gnb_antenna_gain_dbi = 17

ue_transmit_power_dbm = 21
ue_cable_loss_db = 0.5
ue_noise_figure_db = 9
ue_receive_sensitivity_dbm = -95
ua_antenna_gain_dbi = 0

In [3]:
gnb_transmit_power_dbm = convert_milli_watt_to_dbm(gnb_transmit_power_milli_watt)

print(f'gNodeB transmit power: {gnb_transmit_power_dbm:.2f} dBm')

gNodeB transmit power: 30.00 dBm


## Target Data Rate for 4K Stream

- The recommend internet connection speed for playing 4K video from Netflix is 15 Mbps
    - https://help.netflix.com/en/node/306

In [4]:
target_data_rate_bps = 15e6

## Target Channel Capacity

$$ \mathrm{SNR} = 2^{\frac{C}{B}} - 1 $$

- https://www.rfwireless-world.com/calculators/channel-capacity-calculator.html

In [5]:
def get_channel_snr(capacity_bps, bandwidth_hz):
    return math.pow(2, capacity_bps / bandwidth_hz) - 1

In [6]:
target_channel_snr = get_channel_snr(target_data_rate_bps, bandwidth_hz)
target_channel_snr_db = convert_ratio_to_db(target_channel_snr)

print(f'target channel SNR: {target_channel_snr:.2f} or {target_channel_snr_db:.2f} dB')

target channel SNR: 0.68 or -1.66 dB


## Thermal Noise

- https://www.everythingrf.com/rf-calculators/noise-power-calculator

In [7]:
def get_ktb_noise_dbm(bandwidth_hz, temperature_k=290):
    k = 1.38e-23  # Boltzmann constant
    return 10 * math.log10((k * bandwidth_hz * temperature_k)/1e-3)

ktb_noise_dbm = get_ktb_noise_dbm(bandwidth_hz)

print(f'thermal noise: {ktb_noise_dbm:.2f} dBm')

thermal noise: -100.97 dBm


## Target Receive Power

$$ \mathrm{SNR_{dB}} = P_\mathrm{signal,dBm} - P_\mathrm{noise,dBm} $$
$$ P_\mathrm{signal,dBm} = \mathrm{SNR_{dB}} + P_\mathrm{noise,dBm} $$

In [8]:
target_receive_power_dbm = target_channel_snr_db + ktb_noise_dbm

print(f'target receive power: {target_receive_power_dbm:.2f} dBm')

target receive power: -102.63 dBm


## Include Receiver Sensitivity

In [9]:
new_ue_receive_sensitivity_dbm = ue_receive_sensitivity_dbm + ue_noise_figure_db

print(f'new receive sensitivity: {new_ue_receive_sensitivity_dbm:.2f} dBm')

new receive sensitivity: -86.00 dBm


In [10]:
if target_receive_power_dbm < new_ue_receive_sensitivity_dbm:
    print('WARNING: the target receiver power is below the sensitivity')
    target_receive_power_dbm = new_ue_receive_sensitivity_dbm

print(f'new target receive power: {target_receive_power_dbm:.2f} dBm')

new target receive power: -86.00 dBm


## Free Space Path Loss

- https://www.everythingrf.com/rf-calculators/free-space-path-loss-calculator

In [11]:
antenna_coordinates = [51.225779846347535, 4.400503131269632]
min_site_coordinates = [51.227090, 4.393261]
max_site_coordinates = [51.227990333375544, 4.389441701586266]

In [12]:
from geopy import distance

min_distance_m = distance.distance(antenna_coordinates, min_site_coordinates).m
max_distance_m = distance.distance(antenna_coordinates, max_site_coordinates).m

print(f'min distance: {min_distance_m:.2f} m')
print(f'max distance: {max_distance_m:.2f} m')

min distance: 526.48 m
max distance: 810.88 m


In [13]:
def get_fspl_db(distance_m, frequency_hz):
    return 20 * math.log10(distance_m) + 20 * math.log10(frequency_hz) - 147.55

max_fspl_db = get_fspl_db(max_distance_m, frequency_hz)

print(f'max FSPL: {max_fspl_db:.2f} dB')

max FSPL: 96.65 dB


## Power Gains

In [14]:
power_gains_db = gnb_antenna_gain_dbi + ua_antenna_gain_dbi - gnb_cable_loss_db - ue_cable_loss_db

print(f'power gains: {power_gains_db:.2f} dB')

power gains: 16.00 dB


## Target Transmit Power

$$ P_\mathrm{receive,dBm} = P_\mathrm{gains,dB} - P_\mathrm{FSPL,dB} + P_\mathrm{transmit,dBm} $$
$$ P_\mathrm{transmit,dBm} = P_\mathrm{receive,dBm} - P_\mathrm{gains,dB} + P_\mathrm{FSPL,dB} $$

In [15]:
target_transmit_power_dbm = target_receive_power_dbm - power_gains_db + max_fspl_db
target_transmit_power_milli_watt = convert_dbm_to_milli_watt(target_transmit_power_dbm)

print(f'target transmit power: {target_transmit_power_dbm:.2f} dBm or {target_transmit_power_milli_watt:.2f} mW')

target transmit power: -5.35 dBm or 0.29 mW
